In [3]:
%matplotlib notebook

In [4]:
import numpy as np
from facenet_models import FacenetModel
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.pyplot import text
import skimage.io as io
from PIL import Image
import cv2 as cv
import glob

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

if (torch.cuda.is_available()):
    print("GPU available")

GPU available


In [6]:
from eyemodel import EyeModel

In [9]:
cuda = torch.device('cuda')
print(type(cuda))

<class 'torch.device'>


In [11]:
class CustomImageDataset(Dataset):
    def __init__(self, numpydata, numpylabels, transform=None, target_transform=None):
        self.imgs = numpydata
        self.img_labels = numpylabels
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        label = self.img_labels[idx]
        image = self.imgs[idx]
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [13]:
closed_x = np.empty((1,24,24))
open_x = np.empty((1,24,24))

for filename in glob.glob('train_dataset/closed*/*.*'):
    img = np.array([cv.imread(filename, 0)])/255 + 1

    closed_x = np.concatenate((closed_x, img))

for filename in glob.glob('train_dataset/open*/*.*'):
    img = np.array([cv.imread(filename, 0)])/255 + 1

    open_x = np.concatenate((open_x, img))

closed_x = closed_x[1:]
open_x = open_x[1:]

In [15]:
train_x = np.vstack((closed_x[:int(len(closed_x)*0.8)], open_x[:int(len(open_x)*0.8)]))
valid_x = np.vstack((closed_x[int(len(closed_x)*0.8):], open_x[int(len(open_x)*0.8):]))

train_y = [0 for i in range(int(len(closed_x)*0.8))] + [1 for i in range(int(len(open_x)*0.8))]
valid_y = [0 for i in range(len(closed_x) - int(len(closed_x)*0.8))] + [1 for i in range(len(open_x) - int(len(open_x)*0.8))]
# print(train_y[-10])
# print(train_y)
train_x = torch.tensor(train_x, dtype=torch.float32, device=cuda)
valid_x = torch.tensor(valid_x, dtype=torch.float32, device=cuda)

train_y = torch.tensor(train_y, dtype=torch.float32, device=cuda)
valid_y = torch.tensor(valid_y, dtype=torch.float32, device=cuda)

In [17]:
def accuracy(outputs, labels):
    o = torch.argmax(outputs, dim=1)
#     print(o.shape)
#     print(labels.shape)
    return (sum(o == labels) / len(labels))

In [18]:
model = EyeModel().to(cuda)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
batch_size = 32

data = CustomImageDataset(train_x, train_y)
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.reshape(len(inputs),1,24,24)#.float()
        labels = labels.long()

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 70 == 69:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 70:.3f}')
            running_loss = 0.0
            print(f'[{epoch + 1}, {i + 1:5d}] accuracy: {accuracy(outputs, labels)}')
    
    

print('Finished Training')

[1,    70] loss: 0.682
[1,    70] accuracy: 0.5
[2,    70] loss: 0.559
[2,    70] accuracy: 0.71875
[3,    70] loss: 0.437
[3,    70] accuracy: 0.78125
[4,    70] loss: 0.384
[4,    70] accuracy: 0.78125
[5,    70] loss: 0.327
[5,    70] accuracy: 0.90625
[6,    70] loss: 0.312
[6,    70] accuracy: 1.0
[7,    70] loss: 0.259
[7,    70] accuracy: 0.9375
[8,    70] loss: 0.245
[8,    70] accuracy: 0.84375
[9,    70] loss: 0.225
[9,    70] accuracy: 0.90625
[10,    70] loss: 0.219
[10,    70] accuracy: 0.84375
[11,    70] loss: 0.205
[11,    70] accuracy: 0.96875
[12,    70] loss: 0.186
[12,    70] accuracy: 0.875
[13,    70] loss: 0.178
[13,    70] accuracy: 0.9375
[14,    70] loss: 0.180
[14,    70] accuracy: 0.96875
[15,    70] loss: 0.171
[15,    70] accuracy: 0.875
[16,    70] loss: 0.178
[16,    70] accuracy: 0.90625
[17,    70] loss: 0.152
[17,    70] accuracy: 0.96875
[18,    70] loss: 0.148
[18,    70] accuracy: 0.9375
[19,    70] loss: 0.144
[19,    70] accuracy: 0.875
[20,    7

In [19]:
torch.save(model, "model.pb")

In [20]:
model = torch.load("model.pb")

criterion = nn.CrossEntropyLoss()

batch_size = len(valid_x)

valid_data = CustomImageDataset(valid_x, valid_y)
valid_trainloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)

with torch.no_grad():
    running_loss = 0.0
    for i, v_data in enumerate(valid_trainloader, 0):
        v_inputs, v_labels = v_data
        v_inputs = v_inputs.reshape(len(v_inputs),1,24,24)#.float()
        v_labels = v_labels.long()
        
        # forward
        v_outputs = model(v_inputs)
        loss = criterion(v_outputs, v_labels)

        # print statistics
#         running_loss += loss.item()
#         print(f'loss: {running_loss / 2000:.3f}')
        print(f'loss: {loss}')
        print(f'accuracy: {accuracy(v_outputs, v_labels)}')

loss: 0.14131179451942444
accuracy: 0.9536082744598389
